In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow.keras as keras
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#        print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# create a tensorflow training or validation generator function from the tensorflow data generator function, given a directory to interate over
def create_iterators(data_generator, directory, batch_size):
    training_iterator = data_generator.flow_from_directory(
        directory, # directory to get test or train data; must include both classes
        class_mode = 'categorical', # labels generator images based on their directory of origin
        color_mode = 'grayscale',
        batch_size = batch_size,
        subset = 'training'
    )
    validation_iterator = data_generator.flow_from_directory(
        directory, # same directory as training data
        class_mode='categorical',
        color_mode = 'grayscale',
        batch_size = batch_size,
        subset='validation'
    )
    return training_iterator, validation_iterator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator #creates the image data generator object
from keras.models import Sequential #base model for neural network
from keras.layers import InputLayer #input layer
from keras.layers import Conv2D #finds relationships between local features for convolution
from keras.layers import MaxPooling2D #reduces total number of parameters by effectively reducing resolution of image
from keras.layers import Flatten #converts 3D matrix to 2D matrix for final Dense layer
from keras.layers import Dropout #regularization layer
from keras.layers import Dense #final output layer
from tensorflow.keras.optimizers import Adam #momentum + learning rate decay
from keras.losses import CategoricalCrossentropy #loss function #1
from keras.metrics import CategoricalAccuracy #metric function #1
from keras.metrics import AUC #metric function #2
from keras.callbacks import EarlyStopping #save some processing cycles

# design the model - hyperparameters here
def design_model(training_iterator, learning_rate):
    # create neural network model
    model = Sequential()
    # create input layer
    model.add(InputLayer(input_shape = training_iterator.image_shape))
    # add first convolutional layer
    model.add(Conv2D(
        16,
        16,
        strides = 4, # same value for all dimensions in a 2D image
        padding = 'same', #padding with zeros at edge of image
        activation = 'relu' #relu activation
    ))
    # create first max pooling layer
    model.add(MaxPooling2D(
        pool_size = (2,2),
        strides = (2,2),
        padding = 'valid' # no filter padding for max pooling
    ))
    # create first dropout layer
    model.add(Dropout(0.1))
    # create second convolutional layer
    model.add(Conv2D(
        8,
        8,
        strides = 4,
        padding = 'same',
        activation = 'relu'
    ))
    # create second max pooling layer
    model.add(MaxPooling2D(
        pool_size = (2,2),
        strides = (2,2),
        padding = 'valid'
    ))
    # create second dropout layer
    model.add(Dropout(0.1))
    #flatten output for Dense layers
    model.add(Flatten())
    # create hidden layer
    model.add(Dense(8, activation = 'relu'))
    # create output layer
    model.add(Dense(2, activation = 'softmax'))
    
    # create optimization function for model
    opt = Adam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999)
    
    # compile model
    model.compile(
        optimizer = opt,
        loss = CategoricalCrossentropy(),
        metrics = [
            CategoricalAccuracy(),
            AUC()
        ]
    )
    
    # publish model summary, send model back to main body
    model.summary()
    return model
    


In [ ]:
def fit_model(batch_size, learning_rate, num_epochs, weights):
    print("Loading training & validation data\n")
    # create the training generator
    training_data_generator = ImageDataGenerator(
        rescale = 1.0/255, #rescale pixel values from 0 to 255 to 0 to 1 for stability
        zoom_range = 0.1, # database enhancement - zoom by up to 10%
        rotation_range = 15, # database enhancement - rotate up to 15 degrees
        width_shift_range = 0.05, #database enhancement - shift stuff by up to 5% width-wise
        height_shift_range = 0.05, #database enhancement - shift stuff by up to 5% height-wise
        validation_split = 0.2 #create validation generator as well
    )
    # get the training & validation iterators
    training_iterator, validation_iterator = create_iterators(
        training_data_generator, 
        '/kaggle/input/nca-computer-vision-project',
        batch_size
    )

    print("Designing model\n")
    # create the model
    model = design_model(training_iterator, learning_rate)
    # create an early stopping protocol just in case
    stop = EarlyStopping(
        monitor = 'val_loss', # monitor the validation loss to figure out when to stop training
        mode = 'min', # seeks loss minimization
        verbose = 1,
        patience = 10 # number of epochs after validation loss stops getting better before it gives up
    )
    
    print("Training model\n")
    # fit the model
    history = model.fit(
        training_iterator, # the training data
        steps_per_epoch = training_iterator.samples/batch_size,
        epochs = num_epochs, # the number of epochs
        verbose = 1, 
        callbacks = [stop], # add the early stopping protocol
        validation_data = validation_iterator, # the test data
        validation_steps = validation_iterator.samples/batch_size,
        class_weight = weights
    )
    
    return history, training_iterator, validation_iterator

In [ ]:
from tensorflow.keras import Model
from tensorflow import argmax
from tensorflow import expand_dims

# visualizes convolutional layer activations
def visualize_activations(model, validation_iterator):
    # output the fitted model's activations for convolutional layers, but not any other layers
    activation_extractor = Model(
        inputs = model.inputs,
        outputs = [layer.output for layer in model.layers if "conv2d" in layer.name]
    )
    
    # remove axes from plots to make them look cleaner
    def clean_plot(plot):
        plot.axes.get_xaxis().set_visible(False)
        plot.axes.get_yaxis().set_visible(False)
    
    # create a dict mapping labels to categories - note: best guess
    class_names = {
        0:'High G',
        1:'Low G'
    }
    
    # grab 5 images with labels from validation iterator
    sample_batch_input, sample_labels = validation_iterator.next()
    sample_batch_input = sample_batch_input[0:5]
    sample_labels = sample_labels[0:5]
    
    # make predictions using model
    sample_predictions = model.predict(sample_batch_input)
    
    # iterate over each sample in the sample batch & show activations
    for i, (image, prediction, label) in enumerate(zip(sample_batch_input, sample_predictions, sample_labels)):
        image_name = f'Micrograph #{i}' # change this to file names later

        # get predicted class with highest probability, then ground truth
        predicted_class = argmax(prediction).numpy()
        actual_class = argmax(label).numpy()
        
        # present results
        print(image_name)
        print(f'\tModel Prediction: ({prediction} {class_names[actual_class]})')
        print(f'\tActual Class: ({actual_class} {class_names[actual_class]})')
        print(f'\tCorrect: {predicted_class == actual_class}')
        
        # show micrograph
        sample_image = image
        clean_plot(plt.imshow(
            sample_image[:, :, 0],
            cmap = 'gray'
        ))
        plt.title(image_name + f' Predicted {class_names[predicted_class]}, Actual {class_names[actual_class]}')
        plt.tight_layout()
        plt.show()
        model_layer_output = activation_extractor(expand_dims(sample_image, 0))
        plt.clf()
        
        # show the filters
        for l_num, output_data in enumerate(model_layer_output):
            # create a subplot for each filter
            fig, axs = plt.subplots(1, output_data.shape[-1])
            # for each filter
            for i in range(output_data.shape[-1]):
                clean_plot(
                    axs[i].imshow(
                        output_data[0][:, :, i],
                        cmap = 'gray'
                    )
                )
        
        plt.suptitle(image_name + f' Conv {l_num}', y = 0.6)
        plt.tight_layout()
        plt.show()
        plt.clf()
        

In [ ]:
# visualize the accuracy reports
def visualize_accuracy(history):
    # visualize test and train accuracy over time
    fig = plt.figure()
    ax1 = fig.add_subplot(2,1,1)
    ax1.plot(history.history['categorical_accuracy'], label = 'Train')
    ax1.plot(history.history['val_categorical_accuracy'], label = 'Test')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Number of Epochs')
    ax1.set_ylabel('Categorical Accuracy')
    ax1.legend()
    
    # plot auc and validation auc over time
    keys = list(history.history.keys())
    ax2 = fig.add_subplot(2, 1, 2)
    ax2.plot(history.history[keys[2]], label = 'Train')
    ax2.plot(history.history[keys[-1]], label = 'Test')
    ax2.set_title('Model AUC')
    ax2.set_xlabel('Number of Epochs')
    ax2.set_ylabel('AUC')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# create a report of the classes
def report(model, validation_iterator):
    test_steps_per_epoch = np.math.ceil(validation_iterator.samples/validation_iterator.batch_size)
    predictions = model.predict(validation_iterator, steps = test_steps_per_epoch)
    predicted_classes = np.argmax(predictions, axis = 1)
    true_classes = validation_iterator.classes
    class_labels = list(validation_iterator.class_indices.keys())
    
    print(class_labels)
    report = classification_report(true_classes, predicted_classes, target_names = class_labels)
    print(report)
    
    cm = confusion_matrix(true_classes, predicted_classes)
    print(cm)

In [ ]:
BATCH_SIZE = 8
LEARNING_RATE = 0.001
EPOCHS = 50
WEIGHTS = {0:1,1:1.65} # weights for different classes

history, training_iterator, validation_iterator = fit_model(BATCH_SIZE, LEARNING_RATE, EPOCHS, WEIGHTS)

model = history.model
visualize_activations(model, validation_iterator)
visualize_accuracy(history)
report(model, validation_iterator)